# Python implementation of Bayesian Knowledge Tracing (BKT)

Benjamin Xie
University of Washington
bxie@uw.edu

For Codeitz study

Modifications to BKT:
* KT-IDEM: Made guess and slip parameters part of item (not just part of skill/concept) so items are not homogenous (have "difficulty"). From Pardos & Heffernan 2011.
* Sequencing algorithm: Using algorithm to select items. From David, Avi, & Ya'Akov 2016.

## Terminology
* item/exercise: unit representing a task which user does and gets scored. Each item maps to exactly _one_ concept. Each item has 2 parameters: slip and guess probabilities.
* user: learner using Codeitz. A user attempts 0, 1, or many items. They have response data and probability of knowing a certain concept associated with them.
* concept/skill: unit representing latent construct that user could learn. one or many items map to a concept. Each concept has 2 parameters: inital learning and transfer probabilities.

## Notes
* Code documentation roughly follow [NumPy style](https://sphinxcontrib-napoleon.readthedocs.io/en/latest/example_numpy.html)

## References
* David, Yossi Ben, Avi Segal, and Ya’akov (kobi) Gal. 2016. “Sequencing Educational Content in Classrooms Using Bayesian Knowledge Tracing.” In Proceedings of the Sixth International Conference on Learning Analytics & Knowledge, 354–63. ACM.
* Pardos, Zachary A., and Neil T. Heffernan. 2011. “KT-IDEM: Introducing Item Difficulty to the Knowledge Tracing Model.” In User Modeling, Adaption and Personalization, 243–54. Springer Berlin Heidelberg.

In [3]:
import numpy as np
import pandas as pd

## Constants

In [18]:
# params
INIT = 'init'
TRANSFER = 'transfer'
SLIP = 'slip'
GUESS = 'guess'

# concepts/skills
VAR = 'variable'
CONDITIONAL = 'conditional'

# terms
CONCEPT = 'concept'
EID = "eid"
UID = 'uid'
STEP = 'step'
CORRECT = 'correct'
IS_READ = 'is_read'

## Example Data

In [91]:
# making example concepts
c_init = pd.Series([0.1, 0.05, 0.3, 0.2])
c_transfer = pd.Series([0.3, 0.2, 0.3, 0.25])
c_concepts = pd.Series([CONDITIONAL, CONDITIONAL, VAR, VAR])
c_read = pd.Series([True, False, True, False])

df_concepts = pd.DataFrame({CONCEPT: c_concepts, IS_READ: c_read, INIT:c_init, TRANSFER:c_transfer})
print('df_concepts')
print(df_concepts)

# making example item (exercise)
ex_ids = pd.Series(['if/else', 'read vars', 'write vars', 'more read vars', 'read var unused'])
ex_slips = pd.Series([0.2, 0.05, 0.1, 0.2, 0.15])
ex_guesses = pd.Series([0.05, 0.2, 0.1, 0.1, 0.11])
ex_concepts = pd.Series([CONDITIONAL, VAR, VAR, VAR, VAR])
ex_is_read = pd.Series([True, True, False, True, True])

# item 0 should be harder, 1 should be easier, 2 in the middle
df_items = pd.DataFrame({EID: ex_ids, SLIP:ex_slips, GUESS:ex_guesses, CONCEPT:ex_concepts, IS_READ: ex_is_read})
print('\ndf_items')
print(df_items)

# making ledger of 
k_ids = pd.Series(['alex', 'sam'])
df_learn = pd.DataFrame(columns=['uid', 'concept', 'step', 'known'])
print('\ndf_learn')
print(df_learn)

# making example responses
opp_uids = pd.Series(['alex', 'sam', 'sam', 'sam'])
opp_eids = pd.Series(['read vars', 'read vars', 'write vars', 'more read vars'])
opp_step = pd.Series([1, 1, 1, 2])
opp_correct = pd.Series([0, 0, 0, 1])

df_opp = pd.DataFrame({'uid':opp_uids, 'eid': opp_eids, 'step': opp_step, 'correct': opp_correct})
print('\ndf_opp')
print(df_opp) # TODO: add timestamp

df_concepts
       concept  is_read  init  transfer
0  conditional     True  0.10      0.30
1  conditional    False  0.05      0.20
2     variable     True  0.30      0.30
3     variable    False  0.20      0.25

df_items
               eid  slip  guess      concept  is_read
0          if/else  0.20   0.05  conditional     True
1        read vars  0.05   0.20     variable     True
2       write vars  0.10   0.10     variable    False
3   more read vars  0.20   0.10     variable     True
4  read var unused  0.15   0.11     variable     True

df_learn
Empty DataFrame
Columns: [uid, concept, step, known]
Index: []

df_opp
    uid             eid  step  correct
0  alex       read vars     1        0
1   sam       read vars     1        0
2   sam      write vars     1        0
3   sam  more read vars     2        1


In [9]:
df_concepts
df_items
df_opp
df_learn

,uid,concept,step,known


## BKT functions w/ item difficulty

In [80]:
# done! (but not fully tested)
def posterior_pknown(is_correct, eid, transfer, item_params, prior_pknown):
    """
    updates BKT estimate of learner knowledge
    
    Parameters
    ----------
    result: boolean
        True if response was correct
    eid: String
        exercise ID
    transfer: float
        transfer probability for concept
    item_params: pd.DataFrame
        slip and guess parameters for each item
    prior_pknown: float
        prior probability user learned this concept (read, write are different concepts)
    """
    if not eid in item_params[EID].unique():
        raise Exception('Given exercise ID not in response data. Return w/ no update. EID is {}'.format(eid))
        return prior_pknown

    posterior = -1.0
    slip = float(item_params[item_params[EID] == eid][SLIP])
    guess = float(item_params[item_params[EID] == eid][GUESS])
    
    if is_correct:
        posterior = (prior_pknown * (1.0 - slip)) / ((prior_pknown * (1 - slip)) + ((1.0-prior_pknown)*guess))
    else:
        posterior = (prior_pknown * slip) / ((prior_pknown * slip) + ((1.0-prior_pknown)*(1.0-guess)))
    
    return (posterior + (1.0-posterior) * transfer)


def pknown_seq(uid, concept, df_opp, concept_params, item_params, is_read=True):
    """
    Predict sequence of probability a concept is known after each step.
    Function not used in real-time, but may be used to batch update pknown (e.g. if concept or exercise params updated)
    
    Parameters
    ----------
    uid: String
        learner/user ID
    concept: String
        concept name
    df_opp: pd.DataFrame
        dataframe which records the correctness of responses for users. columns: uid, eid, step, correct
    concept_params: pd.DataFrame
        concept parameters (concept, init, transfer)
    item_params: pd.DataFrame
        item parameters (eid, slip, guess, concept)
    is_read: Boolean
        True if concept relates to reading, False if it relates to writing
    """
    
    eids = item_params[(item_params[CONCEPT] == concept) & (item_params[IS_READ] == is_read)][EID]
    
    # grab exercise sequence for specific user working on specific concept
    exercise_seq = df_opp[(df_opp[UID] == uid) & (df_opp[EID].isin(eids))]
    
    n_opps = len(exercise_seq) # number exercises attempted
    
    # filtering for 1 concept to update
    concept_params_target = concept_params[(concept_params[CONCEPT] == concept) & (concept_params[IS_READ] == is_read)]
    
    pk = pd.Series(np.zeros(n_opps + 1))    
    pk[0] = float(concept_params_target[INIT])
    
    if(n_opps > 0):
        transfer = float(concept_params_target[TRANSFER])        
        
        for step in range(1,n_opps+1):
            df_step = exercise_seq[exercise_seq[STEP]==step]
            if(len(df_step) != 1):
                raise Exception('Did not find exactly 1 response for user {} for step {}. Found {}'
                                .format(uid, step, len(df_step)))

            is_correct = df_step.iloc[0][CORRECT]
            eid = df_step.iloc[0][EID]
            
            pk[step] = posterior_pknown(is_correct, eid, transfer, item_params, pk[step - 1])
    return pk

In [16]:
# done! (lightly tested)
def pcorrect(pk, slip, guess):
    return (pk * (1.0-slip)) + ((1.0 - pk) * guess)
    
def pcorrect_seq(uid, concept, df_opp, concept_params, item_params, is_read=True):
    """
    Probability of correct responses predicted by BKT.
    
    Parameters
    ----------
    uid: String
        learner/user ID
    concept: String
        concept name
    df_opp: pd.DataFrame
        dataframe which records the correctness of responses for users. columns: uid, eid, step, correct
    concept_params: pd.DataFrame
        concept parameters (concept, init, transfer)
    item_params: pd.DataFrame
        item parameters (eid, slip, guess, concept)
    is_read: Boolean
        True if concept relates to reading, False if it relates to writing    
    """
    eids = item_params[item_params[CONCEPT] == concept][EID]
    
    # grab exercise sequence for specific user working on specific concept
    exercise_seq = df_opp[(df_opp[UID] == uid) & (df_opp[EID].isin(eids))]    
    n_opps = len(exercise_seq) # number exercises attempted

    pk = pknown_seq(uid, concept, df_opp, concept_params, item_params, is_read)
    pc = pd.Series(np.zeros(n_opps))
    for step in range(0,len(pc)):
        eid = exercise_seq.iloc[step][EID]
        slip = float(item_params[item_params[EID] == eid][SLIP])
        guess = float(item_params[item_params[EID] == eid][GUESS])
        pc[step] = pcorrect(pk[step], slip, guess)

    return pc

In [93]:
# done! (lightly tested)
def order_next_questions(exercise_ids, pk, item_params, error = 0.0, penalty = 1.0):
    """
    Order questions based on "most answerable." 
    Exercise IDs and probability of known must be of same concept, either read or write.
    
    Parameters
    ----------
    exercise_ids: list
        list of exercise ids (Strings) for same concept
    pk: float
        probability concept is known
    item_params: pd.DataFrame
        item parameters (eid, slip, guess, concept)
    """
    
    df_output = pd.DataFrame({"eid": exercise_ids, "score": np.zeros(len(exercise_ids))})
    
    # get max and min scores/p(correct)
    for eid in exercise_ids:
        params = item_params[item_params[EID] == eid].iloc[0]
        df_output.loc[df_output[EID] == eid, 'score'] = pcorrect(pk, params[SLIP], params[GUESS])

    min_score = min(df_output['score'])
    max_score = max(df_output['score'])
    target_score = min_score + ((max_score - min_score) * (1 - pk + error))
#     print('target_score: {}'.format(target_score))
    
    df_output['diff'] = abs(df_output['score'] - target_score) * penalty
    
#     print(df_output)
    return df_output.sort_values(by='diff')[EID]   

## Sandbox

In [11]:
eids = df_items[EID].unique()
df_output = pd.DataFrame({"eid": eids, "score": np.zeros(len(eids))})
df_output.loc[df_output['eid']=='if/else', 'score'] = 0.5
df_output['diff'] = df_output['score'] + 1
df_output.sort_values(by='score')
# df.ix[df['id'] == 12, ['uid','gid']] = ['IN','IN-1']

,eid,score,diff
1,read vars,0.0,1.0
2,write vars,0.0,1.0
0,if/else,0.5,1.5


In [168]:
item_params[item_params[EID] == 'read vars'].iloc[0]

eid        read vars
slip            0.05
guess            0.2
concept     variable
Name: 1, dtype: object

In [108]:
result = 0
eid = 'read vars'
transfer = df_concepts[df_concepts[CONCEPT]==VAR][TRANSFER]
item_params = df_items
prior_pknown = df_concepts[df_concepts[CONCEPT]==VAR][INIT]

correct= result==1
posterior = pd.Series(np.zeros(1))
slip = item_params[item_params[EID] == eid][SLIP]
guess = item_params[item_params[EID] == eid][GUESS]

## Testing pknown()

In [78]:
# posterior_pknown(is_correct, eid, transfer, item_params, prior_pknown)
new_prior = posterior_pknown(0, 'read vars',df_concepts[(df_concepts[CONCEPT]==VAR) & df_concepts[IS_READ]][TRANSFER], df_items, df_concepts[(df_concepts[CONCEPT]==VAR) & df_concepts[IS_READ]][INIT])

print(new_prior)
# new_prior_2 = posterior_pknown(0, 'read vars', df_concepts[df_concepts[CONCEPT]==VAR][TRANSFER], df_items, new_prior)
# print(new_prior_2)
# print(posterior_pknown(0, 'read vars', df_concepts[df_concepts[CONCEPT]==VAR][TRANSFER], df_items, new_prior_2))

2    0.318261
dtype: float64


## testing pknown_seq()

In [83]:
uid = 'sam'
concept = 'variable'
item_params = df_items
concept_params = df_concepts
is_read = True
pknown_seq(uid, concept, df_opp, concept_params, item_params, is_read)

0    0.300000
1    0.318261
2    0.852155
dtype: float64

## Testing pcorrect_seq()

In [84]:
pcorrect_seq(uid, concept, df_opp, concept_params, item_params, is_read)

0    0.425000
1    0.354609
2    0.696509
dtype: float64

In [159]:
eids = item_params[item_params[CONCEPT] == concept][EID]
    
# grab exercise sequence for specific user working on specific concept
exercise_seq = df_opp[(df_opp[UID] == uid) & (df_opp[EID].isin(eids))]    
n_opps = len(exercise_seq) # number exercises attempted

pk = pknown_seq(uid, concept, df_opp, concept_params, item_params)
pc = pd.Series(np.zeros(n_opps))
for step in range(0,len(pc)):
    eid = exercise_seq.iloc[step][EID]
    slip = float(item_params[item_params[EID] == eid][SLIP])
    guess = float(item_params[item_params[EID] == eid][GUESS])
    pc[step] = (pk[step] * (1.0-slip)) + ((1.0-pk[step]) * guess)
pc

0    0.350000
1    0.680571
dtype: float64

## Testing order_next_question()

In [94]:
concept = 'variable'
exercise_ids = df_items[(df_items[CONCEPT] == concept) & (df_items[IS_READ] == True)][EID]
pk = 0.51
item_params = df_items
order_next_questions(exercise_ids, pk, item_params)

4    read var unused
3     more read vars
1          read vars
Name: eid, dtype: object

In [89]:
df_items[df_items[CONCEPT] == concept]

,eid,slip,guess,concept,is_read
1,read vars,0.05,0.2,variable,True
2,write vars,0.10,0.1,variable,False
3,more read vars,0.20,0.1,variable,True
